In [30]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import socket

In [31]:
my_url = 'https://finance.yahoo.com/topic/stock-market-news/'
#my_url = 'https://finance.yahoo.com/quote/XOM/news?p=XOM'

headers = {"User-Agent": "Mozilla/5.0"}
cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}

response = Request(my_url, headers=headers)

In [32]:
def get_page(url):
    """Download a webpage and return a beautiful soup doc"""
    response = requests.get(url, headers=headers, timeout=(3.05, 27))
    page_content = response.text 
    doc = BeautifulSoup(page_content)
    return doc

In [33]:
doc = get_page(my_url)
doc.find('title')

<title>Latest Stock Market News</title>

In [34]:
div_tags = doc.find_all('div', {'class': "Ov(h) Pend(44px) Pstart(25px)"})

In [35]:
len(div_tags)

1

In [36]:
#print(div_tags[1])

In [37]:
#print("Source: ", div_tags[1].find('div').text)
#print("Head Line : {}".format(div_tags[1].find('a').text))

In [38]:
def get_news_tags(doc):
    """Get the list of tags containing news information"""
    news_class = "Ov(h) Pend(44px) Pstart(25px)" ## class name of div tag 
    news_list  = doc.find_all('div', {'class': news_class})
    return news_list

In [39]:
BASE_URL = 'https://finance.yahoo.com' #Global Variable 

def parse_news(news_tag):
    """Get the news data point and return dictionary"""
    news_source = news_tag.find('div').text #source
    news_headline = news_tag.find('a').text #heading
    news_url = news_tag.find('a')['href'] #link
    news_content = news_tag.find('p').text #content
    news_image = news_tag.findParent().find('img')['src'] #thumb image
    return { 'source' : news_source,
            'headline' : news_headline,
            'url' : BASE_URL + news_url,
            'content' : news_content,
            'image' : news_image
           }

In [40]:
import pandas as pd

In [41]:
def scrape_yahoo_news(url, path=None):
    """Get the yahoo finance market news and write them to CSV file """
    if path is None:
        path = 'stock-market-news.csv'
        
    print('Requesting html page')
    doc = get_page(url)

    print('Extracting news tags')
    news_list = get_news_tags(doc)

    print('Parsing news tags')
    news_data = [parse_news(news_tag) for news_tag in news_list]

    print('Save the data to a CSV')
    news_df = pd.DataFrame(news_data)
    news_df.to_csv(path, index=None)
    
    #This return statement is optional, we are doing this just analyze the final output 
    return news_df 

In [42]:
YAHOO_NEWS_URL = BASE_URL+'/topic/stock-market-news/'
news_df = scrape_yahoo_news(YAHOO_NEWS_URL)

Requesting html page
Extracting news tags
Parsing news tags
Save the data to a CSV


In [43]:
news_df.head()

,source,headline,url,content,image
0,Business,Asia stocks edge up despite global growth worries,https://finance.yahoo.com/news/asia-stocks-edg...,"Asian equities edged higher on Thursday, propp...",https://s.yimg.com/uu/api/res/1.2/.sKjuCMiYf_1...
